# Scientific computing

Scientific computing requires much more compute power than "normal" programming. It is therefore worthwhile to understand a bit about the behind-the-scenes of Python.

In [1]:
import numpy as np

Let us look behind the curtain on multidimensional arrays

In [2]:
x = np.arange(15, dtype=float).reshape(5, 3)
x

array([[ 0.,  1.,  2.],
       [ 3.,  4.,  5.],
       [ 6.,  7.,  8.],
       [ 9., 10., 11.],
       [12., 13., 14.]])

We can look at the one-dimensional memory structure of this array. To do this, we use the ravel method and say that we should keep `'K'` the same memory order as in RAM.

In [3]:
x.ravel(order='K')

array([ 0.,  1.,  2.,  3.,  4.,  5.,  6.,  7.,  8.,  9., 10., 11., 12.,
       13., 14.])

Let us tranpose it

In [4]:
x.T

array([[ 0.,  3.,  6.,  9., 12.],
       [ 1.,  4.,  7., 10., 13.],
       [ 2.,  5.,  8., 11., 14.]])

And look at the underlying memory structure of the transposed array

In [5]:
np.ravel(x.T, order='K')

array([ 0.,  1.,  2.,  3.,  4.,  5.,  6.,  7.,  8.,  9., 10., 11., 12.,
       13., 14.])

# Be careful with mutables!
## If we assign y to x, then we get the same variable, but with a different name!

In [6]:
y = x

In [7]:
y[2, 2] = -5

In [8]:
y

array([[ 0.,  1.,  2.],
       [ 3.,  4.,  5.],
       [ 6.,  7., -5.],
       [ 9., 10., 11.],
       [12., 13., 14.]])

In [9]:
y is x

True

## This is also true for lists

In [10]:
l = [1, 2, 3, 4]
ll = l
ll[2] = 10

l

[1, 2, 10, 4]

Assignment does not copy the variable!

In [11]:
def f(input_list=[]):
    input_list.append(1)
    print(input_list)
    
f([-1])

[-1, 1]


In [12]:
f()

[1]


In [13]:
f()

[1, 1]


Why is this happening?

We create the list when we define the function, and then the list is used several times

In [14]:
def f(input_list=[]):
    print(id(input_list))
    input_list.append(1)
    print(input_list)

In [15]:
f()

139841745339464
[1]


In [16]:
f()

139841745339464
[1, 1]


In [17]:
f([])

139841739925192
[1]


See, the ID is the same when we use the default input!

# Back to arrays!
## X and X.T share the same memory

In [18]:
y = x.T

In [19]:
x

array([[ 0.,  1.,  2.],
       [ 3.,  4.,  5.],
       [ 6.,  7., -5.],
       [ 9., 10., 11.],
       [12., 13., 14.]])

In [20]:
y[1, 1] = -10

In [21]:
y

array([[  0.,   3.,   6.,   9.,  12.],
       [  1., -10.,   7.,  10.,  13.],
       [  2.,   5.,  -5.,  11.,  14.]])

In [22]:
x

array([[  0.,   1.,   2.],
       [  3., -10.,   5.],
       [  6.,   7.,  -5.],
       [  9.,  10.,  11.],
       [ 12.,  13.,  14.]])

Why is this? When create y, we set it equal to x.T, and, x.T share the same memory as X!

This can be super useful, as we don't need to copy the variable if we compute the cross product matrix

In [23]:
x.T@x

array([[270., 258., 252.],
       [258., 419., 209.],
       [252., 209., 371.]])

In MATLAB, this will copy x once to create x.T, and then perform the multiplication, if x is large, this can be problematic.

**NumPy will avoid copying elements from `x` as long as possible**

In [24]:
y = np.arange(30).reshape(2, 5, 3)

In [25]:
y2 = np.rollaxis(y, 1, 0)
y2.shape

(5, 2, 3)

In [26]:
print(y.ravel('K'))
print(y2.ravel('K'))

[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25 26 27 28 29]
[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25 26 27 28 29]


But a memory copy will come now:

In [27]:
y2.reshape(5, -1)

array([[ 0,  1,  2, 15, 16, 17],
       [ 3,  4,  5, 18, 19, 20],
       [ 6,  7,  8, 21, 22, 23],
       [ 9, 10, 11, 24, 25, 26],
       [12, 13, 14, 27, 28, 29]])

Which we can see from the timings

In [28]:
%timeit y2.reshape(5, -1)

730 ns ± 145 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


But the original y array had a memory layout that allows us to reshape without copying!

In [29]:
%timeit y.reshape(2, -1)

304 ns ± 75.3 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


This creates a possibility of some weird bugs, where you might think that something is copied but it's not, or the other way around. Most likely, you will not encounter this problem!

# Copying arrays
If you want to force a copy with numpy array, then we can use the `copy` method.

In [30]:
z = y.copy()

In [31]:
z[0, 0, 0] = 10

In [32]:
z[0, 0, 0]

10

In [33]:
y[0, 0, 0]

0

## Copying Python builtins (like lists)
To copy general python objects, use the copy module

In [34]:
import copy

In [35]:
l = [1, 2, [1, 2,]]

In [36]:
l_copy = copy.copy(l)

In [37]:
l_copy[1] = 'hei'

In [38]:
l_copy[1]

'hei'

In [39]:
l[1]

2

This will, however, not recursively copy files!

In [40]:
l_copy[2][1] = 'hei'

In [41]:
l

[1, 2, [1, 'hei']]

To do that, we must use the deepcopy function

In [42]:
l_deepcopy = copy.deepcopy(l)

In [43]:
l_deepcopy[2][1] = 'Deep copies work well'

In [44]:
l_deepcopy[2]

[1, 'Deep copies work well']

In [45]:
l[2]

[1, 'hei']

# Linear algebra in Python
Linear algebra methods lie in `numpy.linalg`.

In [46]:
X = np.random.standard_normal((100, 5))

In [47]:
np.linalg.norm(X, axis=0)  # Compute the norm of X along the first axis

array([9.437167  , 9.49540159, 9.9122429 , 9.3901707 , 9.75970123])

Writing `np.linalg` often is cumbersome, we can therefore import it with an alias

In [48]:
import numpy.linalg as la

In [49]:
la.norm(X, axis=0)

array([9.437167  , 9.49540159, 9.9122429 , 9.3901707 , 9.75970123])

In [50]:
U, s, Vt = la.svd(X, full_matrices=False)  # economy style SVD

**Let us now look at broadcasting**

Broadcasting is how NumPy deals with singleton axes. First, let us create a new vector

In [51]:
X_mean = X.mean(axis=0)  # Compute the mean along the first axis (columns)
# We call the axis we compute the mean along the "reduction axis".
print(X_mean.shape)

(5,)


But is X_mean a row-vector or a column-vector? To be certain, we can add a singleton axis.

In [52]:
X_mean_row = X_mean[np.newaxis, :]
X_mean_col = X_mean[:, np.newaxis]

In [53]:
X_mean_row.shape

(1, 5)

In [54]:
X_mean_col.shape

(5, 1)

In this case, we want the singleton axis along the reduction axis, we can accomplish this with the keepdims argument

In [55]:
X_mean_row = X.mean(axis=0, keepdims=True)
X_mean_col = X.mean(axis=1, keepdims=True)

Now, for *broadcasting* in action:

In [56]:
X_centered = X - X_mean_row

In [57]:
X_centered.mean(axis=0)

array([ 3.57353036e-17, -4.71844785e-18, -1.77635684e-17, -1.33226763e-17,
        8.61810623e-17])

In [58]:
X_row_centered = X - X_mean_col

In [59]:
X_row_centered.mean(axis=1)

array([ 6.66133815e-17,  0.00000000e+00, -2.22044605e-17,  0.00000000e+00,
        0.00000000e+00,  0.00000000e+00,  4.44089210e-17,  0.00000000e+00,
        3.33066907e-17, -8.88178420e-17, -1.11022302e-17, -4.44089210e-17,
        6.66133815e-17, -1.11022302e-17,  4.44089210e-17, -5.55111512e-17,
       -2.49800181e-17,  2.22044605e-17,  4.44089210e-17,  0.00000000e+00,
        8.88178420e-17, -3.33066907e-17,  4.44089210e-17, -2.22044605e-17,
        0.00000000e+00,  1.11022302e-17, -2.22044605e-17, -2.22044605e-17,
        0.00000000e+00,  0.00000000e+00, -2.22044605e-17, -6.66133815e-17,
        0.00000000e+00,  0.00000000e+00,  0.00000000e+00, -8.88178420e-17,
        4.44089210e-17,  2.22044605e-17,  1.77635684e-16, -2.22044605e-17,
        0.00000000e+00, -4.44089210e-17,  2.22044605e-17,  5.55111512e-18,
        0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  4.44089210e-17,
        8.88178420e-17, -4.44089210e-17,  0.00000000e+00, -1.11022302e-17,
       -2.22044605e-17, -

When axes have a different shape, then it will add new axes at the **END**

In [60]:
X_centered = X - X.mean(axis=0)

In [61]:
X_centered.mean(axis=0)

array([ 3.57353036e-17, -4.71844785e-18, -1.77635684e-17, -1.33226763e-17,
        8.61810623e-17])

In [62]:
X_centered.mean(0)  # Axis is first argument

array([ 3.57353036e-17, -4.71844785e-18, -1.77635684e-17, -1.33226763e-17,
        8.61810623e-17])

### Example: Standardising a dataset

In [63]:
X_centered = X - X.mean(axis=0, keepdims=True)
X_standardised = X_centered / la.norm(X_centered, axis=0, keepdims=True)

In [64]:
la.norm(X_standardised, axis=0)

array([1., 1., 1., 1., 1.])

## Task: Standard Normal Variate De-Trending (SNV)
"Standardising along the second axis"

In [65]:
X_SNV = X - X.mean(axis=1, keepdims=True)
X_SNV /= la.norm(X_SNV, axis=1, keepdims=True)

**Task: First SNV then standardise**

In [66]:
X_SNV_centered = X - X.mean(0)
X_SNV_standardised = X_SNV_centered / la.norm(X_SNV_centered, axis=0)

# Reusing scientific code vs creating your own
Why should you use NumPy instead of simply create your own linear algebra library from scratch?

 * NumPy uses the absolute fastest LA code in the world (MKL) (If you install it via anaconda that is)
 * You will make mistakes, there are lots of weird tricks used to make large-scale linear algebra work on computers with finite precision numbers.
 * It is the same with many other packages.

**Example, summing along an axis**

In [67]:
X = np.random.standard_normal((1000, 1000))

In [68]:
%timeit X.mean(0)

973 µs ± 590 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [69]:
%timeit X.mean(1)

451 µs ± 114 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


What's happening here? When we iterate over the first axis, then the data points are not adjacent in memory, which means that it takes a longer time to fetch them from RAM to the CPU.

This is only one of the many things to have in mind when creating such libraries! You don't want to think about this...